## CSE 802 Project: Genre recognition using FMA
Kevin McMahon

In [1]:
%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
tracks = pd.read_csv('fma_metadata/tracks.csv')
genres = pd.read_csv('fma_metadata/genres.csv')
echonest = pd.read_csv('fma_metadata/echonest.csv')
# features = pd.read_csv('fma_metadata/features.csv')

C:\Users\Kevin\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3020: DtypeWarning: Columns (0,1,5,6,8,12,18,20,21,22,24,33,34,38,39,44,47,49) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\Users\Kevin\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3020: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,11,13,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,1

In [3]:
echonest.shape, genres.shape, tracks.shape

((13132, 250), (163, 5), (106576, 53))

In [4]:
genres.loc[:,['title','#tracks']].head()

,title,#tracks
0,Avant-Garde,8693
1,International,5271
2,Blues,1752
3,Jazz,4126
4,Classical,4106


### Cleaning up data
Eliminate first few rows which contain headers and features from echonest that are NaN

In [5]:
echonest.head()

,Unnamed: 0,echonest,echonest.1,echonest.2,echonest.3,echonest.4,echonest.5,echonest.6,echonest.7,echonest.8,...,echonest.239,echonest.240,echonest.241,echonest.242,echonest.243,echonest.244,echonest.245,echonest.246,echonest.247,echonest.248
0,NaN,audio_features,audio_features,audio_features,audio_features,audio_features,audio_features,audio_features,audio_features,metadata,...,temporal_features,temporal_features,temporal_features,temporal_features,temporal_features,temporal_features,temporal_features,temporal_features,temporal_features,temporal_features
1,NaN,acousticness,danceability,energy,instrumentalness,liveness,speechiness,tempo,valence,album_date,...,214,215,216,217,218,219,220,221,222,223
2,track_id,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2,0.4166752327,0.6758939853,0.6344762684,0.0106280683,0.1776465712,0.1593100648,165.9220000000,0.5766609880,NaN,...,-1.9923025370,6.8056936264,0.2330697626,0.1928800046,0.0274549890,0.0640799999,3.6769599915,3.6128799915,13.3166904449,262.9297485352
4,3,0.3744077685,0.5286430621,0.8174611317,0.0018511032,0.1058799438,0.4618181276,126.9570000000,0.2692402421,NaN,...,-1.5823311806,8.8893079758,0.2584637702,0.2209050059,0.0813684240,0.0641300008,6.0827698708,6.0186400414,16.6735477448,325.5810852051


In [6]:
echonest = echonest[3:]
echonest.iloc[:, 7:23].head()

,echonest.6,echonest.7,echonest.8,echonest.9,echonest.10,echonest.11,echonest.12,echonest.13,echonest.14,echonest.15,echonest.16,echonest.17,echonest.18,echonest.19,echonest.20,echonest.21
3,165.9220000000,0.5766609880,NaN,NaN,32.6783000000,"Georgia, US",-83.2230000000,AWOL,AWOL - A Way Of Life,NaN,NaN,NaN,NaN,NaN,0.3889898648,0.3867400000
4,126.9570000000,0.2692402421,NaN,NaN,32.6783000000,"Georgia, US",-83.2230000000,AWOL,AWOL - A Way Of Life,NaN,NaN,NaN,NaN,NaN,0.3889898648,0.3867400000
5,100.2600000000,0.6216612236,NaN,NaN,32.6783000000,"Georgia, US",-83.2230000000,AWOL,AWOL - A Way Of Life,NaN,NaN,NaN,NaN,NaN,0.3889898648,0.3867400000
6,111.5620000000,0.9635898919,2008-03-11,Constant Hitmaker,39.9523000000,"Philadelphia, PA, US",-75.1624000000,Kurt Vile,Constant Hitmaker,2635.0000000000,2544.0000000000,397.0000000000,115691.0000000000,67609.0000000000,0.5573390067,0.6142720000
7,114.2900000000,0.8940722715,NaN,NaN,32.6783000000,"Georgia, US",-83.2230000000,AWOL,AWOL - A Way Of Life,NaN,NaN,NaN,NaN,NaN,0.3889898648,0.3867400000


In [7]:
feature_space = echonest.drop(echonest.columns[9:21],axis=1);
feature_space = feature_space.rename(columns={echonest.columns[0]: 'Track number'})
feature_space.head()

,Track number,echonest,echonest.1,echonest.2,echonest.3,echonest.4,echonest.5,echonest.6,echonest.7,echonest.20,...,echonest.239,echonest.240,echonest.241,echonest.242,echonest.243,echonest.244,echonest.245,echonest.246,echonest.247,echonest.248
3,2,0.4166752327,0.6758939853,0.6344762684,0.0106280683,0.1776465712,0.1593100648,165.9220000000,0.5766609880,0.3889898648,...,-1.9923025370,6.8056936264,0.2330697626,0.1928800046,0.0274549890,0.0640799999,3.6769599915,3.6128799915,13.3166904449,262.9297485352
4,3,0.3744077685,0.5286430621,0.8174611317,0.0018511032,0.1058799438,0.4618181276,126.9570000000,0.2692402421,0.3889898648,...,-1.5823311806,8.8893079758,0.2584637702,0.2209050059,0.0813684240,0.0641300008,6.0827698708,6.0186400414,16.6735477448,325.5810852051
5,5,0.0435668989,0.7455658702,0.7014699916,0.0006967990,0.3731433124,0.1245953419,100.2600000000,0.6216612236,0.3889898648,...,-2.2883579731,11.5271091461,0.2568213642,0.2378199995,0.0601223968,0.0601399988,5.9264898300,5.8663496971,16.0138492584,356.7557373047
6,10,0.9516699648,0.6581786543,0.9245251615,0.9654270154,0.1154738842,0.0329852191,111.5620000000,0.9635898919,0.5573390067,...,-3.6629877090,21.5082283020,0.2833518982,0.2670699954,0.1257044971,0.0808200017,8.4140100479,8.3331899643,21.3170642853,483.4038085938
7,134,0.4522173071,0.5132380502,0.5604099311,0.0194426943,0.0965666940,0.5255193792,114.2900000000,0.8940722715,0.3889898648,...,-1.4526963234,2.3563981056,0.2346863896,0.1995500028,0.1493317783,0.0644000024,11.2670698166,11.2026700974,26.4541797638,751.1477050781


The resulting DataFrame contains 237 features for each track, which should be plently for adequete analysis